In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import sys
import gc
import os
sys.path.append('..')
from train_trl import TrainerWrapper, WrapperConfig, LLAMA_3_2_1B, SMOL_LM_135M
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
project_dir = os.path.abspath(os.path.join(".", os.pardir))
print(project_dir)
cfg = WrapperConfig(
    single_process_mode=True,
    model_id=LLAMA_3_2_1B,
    using_filtered_logprobs=False,
    root_dir=project_dir
)

In [ ]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

In [ ]:
wrapper.init_data_module(False)

In [ ]:
wrapper.tokenizer.eos_token, wrapper.tokenizer.pad_token

In [ ]:
wrapper.init_trainer()

In [ ]:
wrapper.model

In [11]:
gc.collect()
torch.cuda.empty_cache()
! echo $CUDA_VISIBLE_DEVICES
! nvidia-smi

6


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Wed Dec  4 03:38:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:53:00.0 Off |                    0 |
| N/A   52C    P0             399W / 700W |  74349MiB / 81559MiB |     44%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
outputs = wrapper.compute_loss_metrics(1)
outputs = pd.DataFrame(outputs)
outputs.to_parquet("codecontests_dpo.parquet")

In [8]:
wrapper.train()

2024-12-04 03:36:05.422 | INFO     | train_trl:train:227 - Training model
Train dataset reference log probs: 100%|██████████| 103/103 [01:19<00:00,  1.30it/s]
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bfitzgerald. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.20 GiB. GPU 0 has a total capacity of 79.11 GiB of which 5.36 GiB is free. Including non-PyTorch memory, this process has 73.74 GiB memory in use. Of the allocated memory 56.27 GiB is allocated by PyTorch, and 16.75 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
out_df = pd.read_parquet('dpo_scores.parquet')
out_df.head()

out_df

In [ ]:
# plot distribution of losses

STR_COLS = ['prompt', 'chosen', 'rejected']
ZERO_COLS = ['reward_accuracy', 'reward_margin', 'chosen_rewards', 'rejected_rewards', "loss"]

plot_cols = [col for col in out_df.columns if col not in STR_COLS + ZERO_COLS]

fig, axs = plt.subplots(1, len(plot_cols), figsize=(25, 5))

for i, col in enumerate(plot_cols):
    if col in STR_COLS or col in ZERO_COLS:
        continue
    axs[i].hist(out_df[col], bins=50)
    axs[i].set_title(col)

plt.show()

In [ ]:
print(plot_cols)
logprob_differences = out_df['chosen_logps'] - out_df['rejected_logps']
out_df['logprob_differences'] = logprob_differences
plt.hist(logprob_differences, bins=50)
plt.title('logprob_differences')

In [ ]:
from IPython.display import Markdown
samples_sorted_logprob_diff = out_df.sort_values(
    "logprob_differences", ascending=False
)
samples_sorted_highest_diff = samples_sorted_logprob_diff[["prompt", "chosen", "rejected", "logprob_differences"]].head(10)
for prompt, chosen, rejected, logprob_diff in samples_sorted_highest_diff.values:
    display(Markdown(f"\n### Prompt: {prompt}\n\n### Chosen:\n {chosen}\n\n### Rejected:\n {rejected}\n\nLogprob diff: {logprob_diff}"))

In [ ]:
from datasets import Dataset
Dataset.from_pandas(out_df).to_parquet('dpo_scores_sorted.parquet')